## 데이터 읽어오기

In [2]:
import glob
import os,re
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 데이터 정제하기

In [3]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 패스
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 패스

    if idx > 9: break   # 문장 10개 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof


In [4]:
def preprocess_sentence(sentence):
    #소문자로 바꾸고 양쪽 공백 삭제
    sentence = sentence.lower().strip()   
    # 특수문자 양쪽에 공백 추가
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) 
    # 여러 공백을 하나의 공백으로 바꿈
    sentence = re.sub(r'[" "]+', " ", sentence) 
    # a-zA-Z?.!,¿ 을 제외한 모든 문자를 하나의 공백으로 바꿈
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) 
    # 양쪽 공백 삭제
    sentence = sentence.strip()
    # 문장 시작에는 <start>, 문장 끝에는 <end>를 추가
    sentence = '<start> ' + sentence + ' <end>'      
    
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))   

<start> this is sample sentence . <end>


In [5]:
# 여기에 정제된 문장을 모음
corpus = []

for sentence in raw_corpus:
    # 이전처럼 우리가 원하지 않는 문장은 pass
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    
    
    # 정제를 list에 추가
    preprocessed_sentence = preprocess_sentence(sentence)
    #길이 15이상인 토큰 제외
    if len(preprocessed_sentence.split()) > 15: continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

## 데이터 토큰화

In [6]:
def tokenize(corpus):
    
    
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=15000, # 15000단어를 기억할 수 있는 tokenizer 생성
        filters=' ',    # 우리는 필터를 사용하지 않아도 됨
        oov_token="<unk>" # 15000단어에 포함되지 못한 단어는 '<unk>'로!
    )
    # corpus를 이용해 tokenizer 내부의 사전 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  
    
    print(tensor,tokenizer)
    return tensor, tokenizer
tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f7f9b894340>


In [7]:
# 토큰화된 단어 확인

for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


## 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,
                                                          tgt_input,
                                                          train_size = 0.8,
                                                          random_state= 123)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (124810, 14)
Target Train: (124810, 14)


## 데이터셋 객체 생성

In [14]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 15000개와, 여기 포함되지 않은 0:<pad>를 포함하여 15001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset1 = tf.data.Dataset.from_tensor_slices((enc_train, dec_train)).shuffle(BUFFER_SIZE)
dataset1 = dataset1.batch(BATCH_SIZE, drop_remainder=True)
print(dataset1)

dataset2 = tf.data.Dataset.from_tensor_slices((enc_val, dec_val)).shuffle(BUFFER_SIZE)
dataset2 = dataset2.batch(BATCH_SIZE, drop_remainder=True)
print(dataset2)

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>
<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>


## 모델 만들기

In [15]:
#tf.keras.Model 상속해서 사용
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 1024
hidden_size = 2000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [17]:
#  model에 데이터 한 배치만 태워보기
for src_sample, tgt_sample in dataset1.take(1): break
model(src_sample)

<tf.Tensor: shape=(256, 14, 15001), dtype=float32, numpy=
array([[[-6.43394160e-05, -1.80767660e-04, -4.10567518e-05, ...,
         -3.58101752e-05,  3.79382080e-04,  7.90397098e-05],
        [-2.71101540e-04, -1.78351562e-04, -3.25585366e-04, ...,
         -2.93844554e-04,  5.37892687e-04,  3.02417408e-04],
        [-1.49225583e-04,  4.89592145e-04, -2.47398653e-04, ...,
         -5.75735059e-04,  4.96912806e-04,  7.21081509e-04],
        ...,
        [ 1.00437237e-03,  1.36546558e-03, -1.09741150e-03, ...,
          1.02055410e-03, -1.80016831e-03,  3.92881018e-04],
        [ 1.07844942e-03,  1.68935815e-03, -1.69803039e-03, ...,
          1.42133562e-03, -2.02901615e-03,  5.05986856e-04],
        [ 1.12037500e-03,  2.00962787e-03, -2.21423968e-03, ...,
          1.72396435e-03, -2.22957600e-03,  6.75101066e-04]],

       [[-6.43394160e-05, -1.80767660e-04, -4.10567518e-05, ...,
         -3.58101752e-05,  3.79382080e-04,  7.90397098e-05],
        [-1.39270953e-04, -3.73566087e-04, -2

In [18]:
# 모델 확인
model.summary()

Model: "text_generator_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      multiple                  15361024  
_________________________________________________________________
lstm_2 (LSTM)                multiple                  24200000  
_________________________________________________________________
lstm_3 (LSTM)                multiple                  32008000  
_________________________________________________________________
dense_1 (Dense)              multiple                  30017001  
Total params: 101,586,025
Trainable params: 101,586,025
Non-trainable params: 0
_________________________________________________________________


In [19]:
# 학습해보기
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset1, epochs=10,validation_data = dataset2)

Epoch 1/10
487/487 [==============================] - 288s 587ms/step - loss: 3.2753 - val_loss: 2.8998
Epoch 2/10
487/487 [==============================] - 286s 588ms/step - loss: 2.7119 - val_loss: 2.6394
Epoch 3/10
487/487 [==============================] - 286s 587ms/step - loss: 2.3677 - val_loss: 2.4479
Epoch 4/10
487/487 [==============================] - 286s 588ms/step - loss: 2.0348 - val_loss: 2.3142
Epoch 5/10
487/487 [==============================] - 287s 589ms/step - loss: 1.7291 - val_loss: 2.2187
Epoch 6/10
487/487 [==============================] - 287s 590ms/step - loss: 1.4688 - val_loss: 2.1661
Epoch 7/10
487/487 [==============================] - 287s 589ms/step - loss: 1.2651 - val_loss: 2.1482
Epoch 8/10
487/487 [==============================] - 288s 590ms/step - loss: 1.1216 - val_loss: 2.1565
Epoch 9/10
487/487 [==============================] - 287s 590ms/step - loss: 1.0370 - val_loss: 2.1790
Epoch 10/10
487/487 [==============================] - 287s 590m

In [21]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [27]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love it when you call me big poppa <end> '

## 회고


이번 노드는 개인적으로 제일 어려웠던 노드였던것 같다.
모델 만드는 과정도 내가 여태까지 해왔던 layer를 쌓는 방식과는 뭔가 달라서 이해하는게 매우 어려웠고,
텍스트 데이터를 벡터화 시키는것도 이해가 아직도 잘 가지 않는 부분이 많다.

아이펠 후반기에 cv와 nlp중 하나를 선택해야 한다면 당연히 nlp를 해야지 라고 생각했었는데,
진짜 너무 어려운것 같다.. 아직은 노드가 초기 단계여서 더 어렵게 느껴지는것 같은데, 그만큼 nlp를 배워보고 싶다는 욕구도 더 커지는것 같다.
